In [1]:
import numpy as np
import pandas as pd
from operator import itemgetter
from copy import deepcopy
import json

from sklearn.model_selection import train_test_split

import draftsimtools as ds

In [2]:
# Load M19 drafts
raw_drafts = ds.load_drafts("../../data/m19_2.csv")

In [3]:
# Here other folks load card lists, but I grab them from json instead
# m19_set = ds.create_set("data/m19_rating.tsv", "data/m19_land_rating.tsv")

with open('../../data/AllSets.json', 'r',encoding='utf-8') as json_data:
    mtgJSON = json.load(json_data)
jsonSubset = mtgJSON['M19']['cards']
thisSet = {card['name'] : card for card in jsonSubset}

# Another (fancier) way to create a list of names + lots of other useful stuff
nameList = pd.DataFrame.from_dict(thisSet, orient='index', 
                                  columns=['colors','rarity','type','convertedManaCost'])
nameList['Name'] = nameList.index                 # We need names as a column, not an index
nameList['index'] = range(len(nameList))
nameList = nameList.set_index('index')     # And we need a normal numerical index
nameList[1:5]

# Process names, then handle weird card names (those with commas)
nameList['Name'] = nameList.Name.str.replace(' ','_')

# This utility method searches for "Name" column in nameList that have commas
nameList, raw_drafts = ds.fix_commas(nameList, raw_drafts) # Returns a tuple, as it updates both

# Prints namelist
nameList[1:5]

,colors,rarity,type,convertedManaCost,Name
index,,,,,
1,[R],common,Sorcery,3.0,Act_of_Treason
2,[W],uncommon,Instant,2.0,Aegis_of_the_Heavens
3,"[U, W]",uncommon,Creature — Human Artificer,4.0,Aerial_Engineer
4,[U],uncommon,Enchantment — Aura,2.0,Aether_Tunnel


In [4]:
# Process the drafts, deconstructing packs (hands) at every turn of every draft
drafts = ds.process_drafts(raw_drafts)

Processing draft: 0.
Processing draft: 10000.
Processing draft: 20000.
Processing draft: 30000.
Processing draft: 40000.
Processing draft: 50000.
Processing draft: 60000.
Processing draft: 70000.
Processing draft: 80000.
Processing draft: 90000.
Processing draft: 100000.


In [8]:
# Splits into (toy) training and test sets. NOTE: For real training, use all drafts.
subset_drafts = drafts[5000:5500]
#train, test = train_test_split(subset_drafts, test_size = 0.4)

In [9]:
# Make sure all cards are listed in the nameList; update if necessary
for iDraft in range(len(subset_drafts)):
    draft = subset_drafts[iDraft]
    for pack in draft:     
        for cardName in pack:
            try:
                pos = nameList[nameList.Name==cardName].index[0]
            except:
                print("---Unrecognized card: ",cardName) # All unrecognized cards here seem to be foil lands
                #  	colors 	rarity 	type 	convertedManaCost 	Name
                nameList = nameList.append({'colors':[],'rarity':'weird','type':'weird',
                                            'convertedManaCost':0,'Name':cardName},ignore_index=True)

In [10]:
# Splits data into training and testing sets.
# This doesn't matter for some bots, but for others it does,
# so we want to make sure we evaluate on the same testing
# set for every bot. 
train, test = train_test_split(subset_drafts, test_size = 0.4)

# Tests the random and the raredrafting bots against each other
bot1 = ds.RandomBot()
bot2 = ds.RaredraftBot(nameList)
bot3 = ds.ClassicBot(nameList)

tester = ds.BotTester(subset_drafts)
tester.evaluate_bots([bot1, bot2, bot3], ["RandomBot", "RaredratBot", "ClassicBot"])
tester.report_evaluations()
tester.write_evaluations()

draft_num      250.500000
pick_num        23.000000
RandomBot        0.224178
RaredratBot      0.436089
ClassicBot       0.445289
dtype: float64
Wrote correct to: exact_correct.tsv
Wrote fuzzy_correct to: fuzzy_correct.tsv
Wrote rank_error to: rank_error.tsv
Wrote card_acc to: card_accuracies.tsv
